In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_parquet("train_dataset_hackaton2023_train.gzip")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Получаем информацию о данных
print(df.info())

In [ ]:
# Добавим новые признаки
df['month'] = df['startdatetime'].dt.month
df['hour'] = df['startdatetime'].dt.hour
df['day_of_week'] = df['startdatetime'].dt.dayofweek

In [ ]:
# Проверить наличие пропусков
missing_values = df.isnull().sum()
print("\nПропуски:")
print(missing_values)

# БАЛАНС КЛАССОВ

In [ ]:
#подсчет количества экземпляров каждого класса в целевой переменной
buy_counts = df['buy_post'].value_counts()
print(buy_counts[0]/buy_counts[1])
print(buy_counts)


# распределение

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Распределение целевого признака date_diff_post
plt.figure(figsize=(12, 6))
sns.histplot(df['date_diff_post'], bins=20, kde=True, color='blue')
plt.title('Distribution of date_diff_post')
plt.xlabel('date_diff_post')
plt.ylabel('Frequency')
plt.show()

# Ящик с усами для целевого признака date_diff_post
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['date_diff_post'])
plt.title('Box plot of date_diff_post')
plt.show()

# Распределение целевого признака buy_post
plt.figure(figsize=(12, 6))
sns.histplot(df['buy_post'], bins=20, kde=True, color='green')
plt.title('Distribution of buy_post')
plt.xlabel('buy_post')
plt.ylabel('Frequency')
plt.show()

# Ящик с усами для целевого признака buy_post
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['buy_post'])
plt.title('Box plot of buy_post')
plt.show()


Распределение целевых признаков предоставляет информацию о структуре данных и их характеристиках.

 медиана в ящике с усами покажет, где находится центр распределения.

Разброс данных:
Размах и длина ящика могут указать на разброс данных. Это может быть полезно для оценки диапазона значений и наличия выбросов.

Выбросы:
Точки за пределами ящика считаются выбросами. Их обнаружение может быть важным при решении о том, следует ли исключить или обработать эти значения.

Форма распределения

Частота значений:
Гистограмма позволяет оценить, как часто встречаются различные значения.

На основе этих аспектов можно принимать решения относительно предобработки данных, выявления выбросов, выбора подходящих методов анализа и построения моделей машинного обучения.

#### Данные не сбалансированы!!
SMOTE? oversample? Веса?

# Блюда в цифры

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# dish_name в числовой формат
df['dish_name'] = label_encoder.fit_transform(df['dish_name'])

print(df.info())

# соответствия чисел и блюд
encoded_dishes = pd.DataFrame({
    'dish_name_encoded': df['dish_name'].unique(),
    'dish_name': label_encoder.inverse_transform(df['dish_name'].unique())
})

print(encoded_dishes)


In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['group_name'] = label_encoder.fit_transform(df['group_name'])
df['format_name'] = label_encoder.fit_transform(df['format_name'])


print(df['group_name'].unique())

print(df['format_name'].unique())
for original, encoded in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{encoded}: {original}")

print(df.info())


In [ ]:
df = df.drop(columns=['group_name'])

# Корреляция

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# корреляци
correlation_m = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_m, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
correlation_m

## ПРОПУСКИ

In [ ]:
median_value = df['date_diff_post'].median()
df['date_diff_post'].fillna(median_value, inplace=True)

In [ ]:
# Проверить наличие пропусков
missing_values = df.isnull().sum()
print("\nПропуски:")
print(missing_values)

# BASELINE 1

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

X = df.drop(['buy_post', 'startdatetime', 'date_diff_post'], axis=1)
y = df['buy_post']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = DecisionTreeClassifier(random_state=42)

# Обучим
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)



In [ ]:
df.info()

Без баланса, и без optuna, метрики:


In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

### Группируем

In [ ]:
import pandas as pd

# Создайте DataFrame с прогнозами и вероятностями
predictions_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],  # Предполагая, что у вас есть столбец 'customer_id'
    'buy_post': y_pred,
})

grouped_predictions = predictions_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()

grouped_predictions['prob_score'] = grouped_predictions[1] / (grouped_predictions[0] + grouped_predictions[1])

grouped_predictions['buy_post'] = grouped_predictions[1].apply(lambda x: 1 if x > 0 else 0)

result_df = grouped_predictions[['customer_id', 'buy_post', 'prob_score']]

print(result_df)

In [ ]:
import pandas as pd

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'buy_post': y_test,
})


grouped_true_values = true_values_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()


grouped_true_values['prob_score'] = grouped_true_values[1] / (grouped_true_values[0] + grouped_true_values[1])


grouped_true_values['buy_post'] = grouped_true_values[1].apply(lambda x: 1 if x > 0 else 0)


real_y = grouped_true_values[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(real_y)


In [ ]:
# Оценим качество
accuracy = accuracy_score(real_y['buy_post'], result_df['buy_post'])
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(real_y['buy_post'], result_df['buy_post'])
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(real_y['buy_post'], result_df['buy_post'])
print(f"ROC-AUC Score: {roc_auc:.2f}")

# Логистическая регрессия без подбора гиперпараметров и без баланса

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd


X = df.drop(['buy_post', 'startdatetime', 'date_diff_post'], axis=1)
y = df['buy_post']

#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LogisticRegression(random_state=42)


model.fit(X_train, y_train)


y_pred = model.predict(X_test)



In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
y_pred

In [ ]:
predictions_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],  # Предполагая, что у вас есть столбец 'customer_id'
    'buy_post': y_pred,
})

grouped_predictions = predictions_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()

grouped_predictions['prob_score'] = grouped_predictions[1] / grouped_predictions[1].sum()

grouped_predictions['buy_post'] = grouped_predictions[1].apply(lambda x: 1 if x > 0 else 0)


result_df = grouped_predictions[['customer_id', 'buy_post', 'prob_score']]


print(result_df)

In [ ]:
import pandas as pd

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'buy_post': y_test,
})


grouped_true_values = true_values_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()


grouped_true_values['prob_score'] = grouped_true_values[1] / (grouped_true_values[0] + grouped_true_values[1])


grouped_true_values['buy_post'] = grouped_true_values[1].apply(lambda x: 1 if x > 0 else 0)


real_y = grouped_true_values[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(real_y)


In [ ]:
# Оценим качество
accuracy = accuracy_score(real_y['buy_post'], result_df['buy_post'])
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(real_y['buy_post'], result_df['buy_post'])
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(real_y['buy_post'], result_df['buy_post'])
print(f"ROC-AUC Score: {roc_auc:.2f}")

# Баланс классов


In [1]:
# from imblearn.over_sampling import KMeansSMOTE
# from sklearn.model_selection import train_test_split

# X = df.drop(['buy_post', 'startdatetime'], axis=1)
# y = df['buy_post']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# kmeans_smote = KMeansSMOTE(cluster_balance_threshold=0.1, random_state=42)
# X_train_resampled, y_train_resampled = kmeans_smote.fit_resample(X_train, y_train)

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = df.drop(['buy_post', 'startdatetime', 'date_diff_post'], axis=1)
y = df['buy_post']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# ЛОГРЕГ с балансом и оптуной

In [ ]:
%%time
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import pandas as pd
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

def objective(trial):
    C = trial.suggest_float("C", 0.001, 1000)
    penalty = trial.suggest_categorical("penalty", ["l2"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter,
        random_state=42
    )

    # Обучение
    model.fit(X_resampled, y_resampled)

    # Предсказание
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter,
    random_state=42
)

best_model.fit(X_resampled, y_resampled)

y_pred = best_model.predict(X_test)


In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

# ДЕРЕВО С БАЛАНСОМ И ОПТУНОЙ

In [ ]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

def objective(trial):
    # Определите пространство поиска гиперпараметров
    max_depth = trial.suggest_int("max_depth", 3, 15)
    min_samples_split = trial.suggest_float("min_samples_split", 0.1, 1.0)
    min_samples_leaf = trial.suggest_float("min_samples_leaf", 0.1, 0.5)

    # Модель решающего дерева
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Создание объекта study и запуск оптимизации
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Лучшие гиперпараметры
best_params = study.best_params
best_max_depth = best_params["max_depth"]
best_min_samples_split = best_params["min_samples_split"]
best_min_samples_leaf = best_params["min_samples_leaf"]

# Модель с лучшими гиперпараметрами
best_model = DecisionTreeClassifier(
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    min_samples_leaf=best_min_samples_leaf,
    random_state=42
)

best_model.fit(X_resampled, y_resampled)

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)


In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
import pandas as pd

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'buy_post': y_test,
})


grouped_true_values = true_values_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()


grouped_true_values['prob_score'] = grouped_true_values[1] / (grouped_true_values[0] + grouped_true_values[1])


grouped_true_values['buy_post'] = grouped_true_values[1].apply(lambda x: 1 if x > 0 else 0)


real_y = grouped_true_values[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(real_y)

In [ ]:
import pandas as pd

# DataFrame с прогнозами и вероятностями
predictions_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],  # Предполагая, что у вас есть столбец 'customer_id'
    'buy_post': y_pred,
})

# Сгруппир по 'customer_id' и подсчит количество 0 и 1 для каждого пользователя
grouped_predictions = predictions_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()

# вероятность как отношение количества 1 к общему количеству
grouped_predictions['prob_score'] = grouped_predictions[1] / (grouped_predictions[0] + grouped_predictions[1])

# buy_post' принимает только два значения
grouped_predictions['buy_post'] = grouped_predictions[1].apply(lambda x: 1 if x > 0 else 0)

result_df = grouped_predictions[['customer_id', 'buy_post', 'prob_score']]

print(result_df)

In [ ]:
# Оценим качество
accuracy = accuracy_score(real_y['buy_post'], result_df['buy_post'])
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(real_y['buy_post'], result_df['buy_post'])
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(real_y['buy_post'], result_df['buy_post'])
print(f"ROC-AUC Score: {roc_auc:.2f}")

# CatBoost с подбором и после Smote

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 500),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0),
        'border_count': trial.suggest_int('border_count', 5, 200),
    }

    model = CatBoostClassifier(**params, random_state=42, silent=True)

    # Обучение
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=False)

    # Предсказание
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Создание объекта study и запуск оптимизации
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Лучшие гиперпараметры
best_params = study.best_params
best_iterations = best_params["iterations"]
best_depth = best_params["depth"]
best_learning_rate = best_params["learning_rate"]
best_l2_leaf_reg = best_params["l2_leaf_reg"]
best_border_count = best_params["border_count"]

# Модель с лучшими гиперпараметрами
best_model = CatBoostClassifier(
    iterations=best_iterations,
    depth=best_depth,
    learning_rate=best_learning_rate,
    l2_leaf_reg=best_l2_leaf_reg,
    border_count=best_border_count,
    random_state=42,
    silent=True
)

best_model.fit(X_resampled, y_resampled)

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)


In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
# Важность признаков
feature_importances = best_model.feature_importances_
print(feature_importances)
print(X.columns)
# Визуализация важности признаков
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances, align='center')
plt.xticks(range(len(feature_importances)), X.columns, rotation=45)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Catboost')
plt.show()

In [ ]:
import pandas as pd

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'buy_post': y_test,
})


grouped_true_values = true_values_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()


grouped_true_values['prob_score'] = grouped_true_values[1] / (grouped_true_values[0] + grouped_true_values[1])


grouped_true_values['buy_post'] = grouped_true_values[1].apply(lambda x: 1 if x > 0 else 0)


real_y = grouped_true_values[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(real_y)

In [ ]:
import pandas as pd

# DataFrame с прогнозами и вероятностями
predictions_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],  # Предполагая, что у вас есть столбец 'customer_id'
    'buy_post': y_pred,
})

# Сгрупп по 'customer_id' и подсчит количество 0 и 1 для каждого пользователя
grouped_predictions = predictions_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()

# вероятность как отношение количества 1 к общему количеству
grouped_predictions['prob_score'] = grouped_predictions[1] / (grouped_predictions[0] + grouped_predictions[1])

# 'buy_post' принимает только два значения
grouped_predictions['buy_post'] = grouped_predictions[1].apply(lambda x: 1 if x > 0 else 0)

# только нужные столбцы
result_df = grouped_predictions[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(result_df)

In [ ]:
# Оценим качество
accuracy = accuracy_score(real_y['buy_post'], result_df['buy_post'])
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(real_y['buy_post'], result_df['buy_post'])
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(real_y['buy_post'], result_df['buy_post'])
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
fpr4, tpr4, tresholds4 = roc_curve(real_y['buy_post'], result_df['buy_post'])

roc_auc4 = auc(fpr4, tpr4)

#plt.plot(fpr, tpr, color='lime', label='ROC curve BS(area = %0.2f)' % roc_auc)
# plt.plot(fpr_2, tpr_2, color='red', label='ROC curve GR (area = %0.2f)' % roc_auc_2)
#plt.plot(fpr3, tpr3, color='orange', label='ROC curve CB (area = %0.2f)' % roc_auc3)
plt.plot(fpr4, tpr4, color='blue', label='ROC curve CB_GR (area = %0.2f)' % roc_auc4)
plt.plot([0, 1], [0, 1], color = 'navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC AUC')
plt.legend(loc='lower right')
plt.show()

# HIST GRADIENT

In [ ]:
import optuna
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

def objective(trial):
    # Определите пространство поиска гиперпараметров
    params = {
        'max_iter': trial.suggest_int('max_iter', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 1000),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-10, 10),
        'max_bins': trial.suggest_int('max_bins', 2, 255),
    }

    # Модель HistGradientBoostingClassifier
    model = HistGradientBoostingClassifier(**params, random_state=42)

    # Обучение
    model.fit(X_train, y_train)

    # Предсказание
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Лучшие гиперпараметры
best_params = study.best_params
best_max_iter = best_params["max_iter"]
best_learning_rate = best_params["learning_rate"]
best_max_depth = best_params["max_depth"]
best_min_samples_leaf = best_params["min_samples_leaf"]
best_max_leaf_nodes = best_params["max_leaf_nodes"]
best_l2_regularization = best_params["l2_regularization"]
best_max_bins = best_params["max_bins"]

# Модель с лучшими гиперпараметрами
best_model = HistGradientBoostingClassifier(
    max_iter=best_max_iter,
    learning_rate=best_learning_rate,
    max_depth=best_max_depth,
    min_samples_leaf=best_min_samples_leaf,
    max_leaf_nodes=best_max_leaf_nodes,
    l2_regularization=best_l2_regularization,
    max_bins=best_max_bins,
    random_state=42
)

best_model.fit(X_resampled, y_resampled)

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test)

In [ ]:
# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
import pandas as pd

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'buy_post': y_test,
})


grouped_true_values = true_values_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()


grouped_true_values['prob_score'] = grouped_true_values[1] / (grouped_true_values[0] + grouped_true_values[1])


grouped_true_values['buy_post'] = grouped_true_values[1].apply(lambda x: 1 if x > 0 else 0)


real_y = grouped_true_values[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(real_y)

In [ ]:
import pandas as pd

# DataFrame с прогнозами и вероятностями
predictions_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],  # Предполагая, что у вас есть столбец 'customer_id'
    'buy_post': y_pred,
})

# Сгруппи по 'customer_id' и подс количество 0 и 1 для каждого пользователя
grouped_predictions = predictions_df.groupby('customer_id')['buy_post'].value_counts().unstack(fill_value=0).reset_index()

# вероятность как отношение количества 1 к общему количеству
grouped_predictions['prob_score'] = grouped_predictions[1] / (grouped_predictions[0] + grouped_predictions[1])

# 'buy_post' принимает только два значения
grouped_predictions['buy_post'] = grouped_predictions[1].apply(lambda x: 1 if x > 0 else 0)

# только нужные столбцы
result_df = grouped_predictions[['customer_id', 'buy_post', 'prob_score']]

# Выведите результаты
print(result_df)

In [ ]:
# Оценим качество
accuracy = accuracy_score(real_y['buy_post'], result_df['buy_post'])
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(real_y['buy_post'], result_df['buy_post'])
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(real_y['buy_post'], result_df['buy_post'])
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, auc, roc_curve, roc_auc_score

fpr4, tpr4, tresholds4 = roc_curve(real_y['buy_post'], result_df['buy_post'])


roc_auc4 = auc(fpr4, tpr4)

#plt.plot(fpr, tpr, color='lime', label='ROC curve BS(area = %0.2f)' % roc_auc)
# plt.plot(fpr_2, tpr_2, color='red', label='ROC curve GR (area = %0.2f)' % roc_auc_2)
#plt.plot(fpr3, tpr3, color='orange', label='ROC curve CB (area = %0.2f)' % roc_auc3)
plt.plot(fpr4, tpr4, color='blue', label='ROC curve CB_GR (area = %0.2f)' % roc_auc4)
plt.plot([0, 1], [0, 1], color = 'navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC AUC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    real_y['buy_post'], result_df['prob_score'],
    color="darkorange",
)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
import joblib

# Сохранение модели
joblib.dump(best_model, 'best_model_hist_churn.joblib')

# Сохранение параметров
with open('best_model_params_ch.txt', 'w') as f:
    f.write(str(best_params))


# catboost для diff_time

In [2]:
#Заново загружаем данные и балансим
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = df.drop(['buy_post', 'startdatetime', 'date_diff_post'], axis=1)
y = df['date_diff_post']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
# X_resampled

In [ ]:
X

In [ ]:
# import optuna
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Разделение данных на обучающий и тестовый наборы
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# def objective(trial):
#     params = {
#     'objective': 'multiclass',
#     'num_classes': 60,  # Замените num_classes на количество уникальных классов в вашем целевом столбце
#     'metric': 'multi_error',
#     'boosting_type': 'gbdt',
#     'num_leaves': trial.suggest_int('num_leaves', 20, 200),
#     'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#     'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
#     'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
#     'lambda_l1': trial.suggest_float('lambda_l1', 1e-9, 10.0, log=True),
#     'lambda_l2': trial.suggest_float('lambda_l2', 1e-9, 10.0, log=True),
#     }


#     model = lgb.LGBMClassifier(**params, random_state=42)

#     # Обучение
#     model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)

#     # Предсказание
#     y_pred = model.predict(X_test)

#     accuracy = accuracy_score(y_test, y_pred)

#     return accuracy

# # Создание объекта study и запуск оптимизации
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# # Лучшие гиперпараметры
# best_params = study.best_params
# best_num_leaves = best_params["num_leaves"]
# best_learning_rate = best_params["learning_rate"]
# best_feature_fraction = best_params["feature_fraction"]
# best_bagging_fraction = best_params["bagging_fraction"]
# best_lambda_l1 = best_params["lambda_l1"]
# best_lambda_l2 = best_params["lambda_l2"]

# # Модель с лучшими гиперпараметрами
# best_model = lgb.LGBMClassifier(
#     objective='binary',
#     metric='binary_error',
#     boosting_type='gbdt',
#     num_leaves=best_num_leaves,
#     learning_rate=best_learning_rate,
#     feature_fraction=best_feature_fraction,
#     bagging_fraction=best_bagging_fraction,
#     lambda_l1=best_lambda_l1,
#     lambda_l2=best_lambda_l2,
#     random_state=42
# )

# best_model.fit(X_resampled, y_resampled)

# # Предсказание на тестовых данных
# y_pred = best_model.predict(X_test)


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',  # Квадратный корень из среднеквадратичной ошибки
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-9, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-9, 10.0, log=True),
    }

    model = lgb.LGBMRegressor(**params, random_state=42)

    # Обучение
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)

    # Предсказание
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

    rmse = mean_squared_error(y_test, y_pred, squared=False)

    return -rmse  # Минимизация RMSE, поэтому используем знак минус

# Создание объекта study и запуск оптимизации
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# Лучшие гиперпараметры
best_params = study.best_params
best_num_leaves = best_params["num_leaves"]
best_learning_rate = best_params["learning_rate"]
best_feature_fraction = best_params["feature_fraction"]
best_bagging_fraction = best_params["bagging_fraction"]
best_lambda_l1 = best_params["lambda_l1"]
best_lambda_l2 = best_params["lambda_l2"]

# Модель с лучшими гиперпараметрами
best_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    boosting_type='gbdt',
    num_leaves=best_num_leaves,
    learning_rate=best_learning_rate,
    feature_fraction=best_feature_fraction,
    bagging_fraction=best_bagging_fraction,
    lambda_l1=best_lambda_l1,
    lambda_l2=best_lambda_l2,
    random_state=42
)

best_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)

# Предсказание на тестовых данных
y_pred = best_model.predict(X_test, num_iteration=best_model.best_iteration_)


In [ ]:
# # Оценим качество
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

# # Вычисление F1-меры
# f1 = f1_score(y_test, y_pred)
# print(f"F1 Score: {f1:.2f}")

# # Вычисление ROC-AUC
# roc_auc = roc_auc_score(y_test, y_pred)
# print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"RMSE: {rmse}")

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)

print(f"MSE: {mae}")

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mae = mean_absolute_percentage_error(y_test, y_pred)

print(f"MSE: {mae}")

In [ ]:
import pandas as pd


predictions_df = pd.DataFrame({'customer_id': X_test['customer_id'], 'predicted_date_diff_post': y_pred})

average_predictions = predictions_df.groupby('customer_id')['predicted_date_diff_post'].mean().reset_index()

# Вывод результатов
print(average_predictions)

In [ ]:
import pandas as pd

# Создание DataFrame с предсказанными значениями
predictions_df = pd.DataFrame({'customer_id': X_test['customer_id'], 'predicted_date_diff_post': y_pred})

# Группировка предсказаний по 'customer_id' и вычисление среднего значения
average_predictions = predictions_df.groupby('customer_id')['predicted_date_diff_post'].mean().reset_index()

# Создайте DataFrame с реальными значениями
true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'date_diff': y_test,
})

# Группировка реальных значений по 'customer_id' и вычисление среднего значения
average_true_values = true_values_df.groupby('customer_id')['date_diff'].mean().reset_index()

# Соединение двух DataFrame для сравнения
comparison_df = pd.merge(average_true_values, average_predictions, on='customer_id', suffixes=('_true', '_pred'))

# Вывод результатов
print(comparison_df)


In [ ]:
from sklearn.metrics import mean_squared_error

# Вычисление RMSE
rmse = mean_squared_error(comparison_df['date_diff'], comparison_df['predicted_date_diff_post'], squared=False)

print(f'Root Mean Squared Error (RMSE): {rmse}')


In [ ]:
import pandas as pd

true_values_df = pd.DataFrame({
    'customer_id': X_test['customer_id'],
    'date_diff_post': y_test,
})

average_true_values = true_values_df.groupby('customer_id')['date_diff_post'].mean().reset_index()

# # Соединение двух DataFrame для сравнения
# comparison_df = pd.merge(average_true_values, average_predictions, on='customer_id', suffixes=('_true', '_pred'))

# # Вывод результатов
# print(comparison_df)


In [ ]:
# true_values_df['date_diff_post']
average_predictions['predicted_date_diff_post']

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"RMSE: {rmse}")

In [ ]:
import joblib

# Сохранение модели
joblib.dump(best_model, 'best_model_light.pkl')

# # Загрузка модели
# loaded_model = joblib.load('best_model.pkl')

# # Предсказание на новых данных
# new_data = pd.DataFrame(...)
# predictions = loaded_model.predict(new_data)


# ПРЕДСКАЗАНИЕ НА ТЕСТЕ

In [ ]:
import pandas as pd
import joblib

# Загрузка тестовых данных
df  = pd.read_parquet('test_dataset_hackaton2023_test.parquet')

# Загрузка модели HistGradientBoostingClassifier
best_model = joblib.load('best_model_hist_churn.joblib')

df

# # Предсказание на тестовых данных
# predictions = best_model.predict(test_data)

# # Сохранение результатов в CSV файл
# results_df = pd.DataFrame({'customer_id': test_data['customer_id'], 'predictions': predictions})
# results_df.to_csv('predictions.csv', index=False)


In [ ]:
df = df.drop('startdatetime', axis=1)

In [ ]:
import joblib

best_model_hist = joblib.load('best_model_hist_churn.joblib')

predictions_buy_post = best_model_hist.predict(df)


In [ ]:
predictions_buy_post

In [ ]:
import joblib

# Загрузка модели LightGBM
best_model_light = joblib.load('best_model_light.pkl')

# Предсказание 'date_diff_post' с использованием модели LightGBM
predictions_date_diff_post = best_model_light.predict(df)

In [ ]:
predictions_date_diff_post

In [ ]:
# Создание DataFrame с предсказаниями
predictions_df = pd.DataFrame({
    'customer_id': df['customer_id'],
    'predicted_buy_post': predictions_buy_post,
    'predicted_date_diff_post': predictions_date_diff_post
})




In [ ]:
import pandas as pd

predictions_df = pd.DataFrame({
    'customer_id': df['customer_id'],
    'predicted_buy_post': predictions_buy_post,  # Предсказания для 'buy_post'
    'predicted_date_diff_post': predictions_date_diff_post  # Предсказания для 'date_diff_post'
})


grouped_buy_post = predictions_df.groupby('customer_id')['predicted_buy_post'].value_counts().unstack(fill_value=0).reset_index()

grouped_buy_post['buy_post'] = grouped_buy_post[[0, 1]].idxmax(axis=1)

grouped_date_diff_post = predictions_df.groupby('customer_id')['predicted_date_diff_post'].mean().reset_index()

result_df = pd.merge(grouped_buy_post[['customer_id', 'buy_post']], grouped_date_diff_post, on='customer_id')

result_df


In [ ]:
result_df.to_csv('submit_k.csv', sep=';', index=False)
